In [1]:
from api.riot_api_routes import *
from api.variables import SERVER_ROUTES, SERVER_REAL_NAME_TO_ROUTE
from api.variables import DIVISIONS, TIERS, GAME_TYPE,MAIN_GAMEMODE,RANKED_QUEUES
from api.variables import MATCHES_STATS_KEYS,PLAYSTYLE_STATS_KEYS

import json
from django.conf import settings
from os import path
import os
import random

import pandas as pd
from time import sleep

league_data_route=path.join(settings.PREPROCESS_PATH, "league_data.json")
player_sample_route=path.join(settings.PREPROCESS_PATH, "player_sample.csv")
matches_sample_route=path.join(settings.PREPROCESS_PATH, "matches_sample.csv")
champ_ban_file=path.join(settings.PREPROCESS_PATH, "champ_ban.csv")
champ_data_file=path.join(settings.PREPROCESS_PATH, "champ_data.csv")
playstyle_data_file=path.join(settings.PREPROCESS_PATH, "playstyle_data.csv")



In [2]:
df = pd.read_csv(player_sample_route)
df

,summonerId,tier,leaguePoints,wins,losses,summonerName,server,division,winrate,accountId,puuid,profileIconId,summonerLevel,revisionDate
0,-cNyrEDWt-wLzgX2R-aJ_yBHi1VGEizbhDzsMhV4PMUU,I,489,81,61,TutePra,LAS,challengers,57.0,G2vtv9B81PqkVkLpIogPHjep6YYqPS0SotMYWgsu4O8,Z3D2_QAsR-fUngZ3z7ZSKQ7X1rb7cvkWF2GvJEVg0w2Cjy...,3861,284,1585775232000
1,QxwEVtn25FF6Q2qhHOFHZgcnD5yksLrE0od01GWg6R7b,I,806,162,136,rolEˆ,LAS,challengers,54.4,pR8Bw7gggybEx0EmsTeRC789lZipvbSAjV7CVtt-THk,otll2Dw6URRD_qkh718UH6uvkoWDLtnYxEAQayi2Tll8KC...,586,267,1586297064000
2,XusjYZoWn5qHcOoQ7WeXqIzvpLanpR3zW2K_7Hfj1W-9,I,517,99,85,TMG Niqqe,LAS,challengers,53.8,ElSY9K8ThKKzDw5WNJK_a6Q1gjbokrVOYqh0fX68Aig,SzkFqAjC_XoeU-YVlyKVbvzVBgP_zG1TYXCTkhWtxPUdu3...,4154,186,1586385206000
3,wH1K-zLvrwbF5gz9k1luFxRV-szDOeOkYQNkBzpd-yQ,I,1079,87,51,Morwinelo,LAS,challengers,63.0,nYHfpOcJYXQ3TFnpDl-VGKtoDP3Zy_cWj7ZAqjoZ1g,rf4w_WsyM3GxcHny65HlwjbLsIYgwD0iABJ2EBlR0I5gf2...,3152,217,1586300756000
4,DVLZaSvdjFjSSyeCe5qkmJYwWMAXibg7NGcgAUY0-qP3lQ,I,496,139,116,EDS ratrenix,LAS,challengers,54.5,ZpuuPRM-AA8tAvJmGTalfO95RSKDNn6-KhM0oe82kp3KGqE,hPtzyohDAbyEGbYdoZa4yx5vXJkQoHY--8AsylTCeLbgsX...,783,243,1586471672000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,CgLM03eoXLu4sUAUOv40Kyd3a524GnEbxLrSrWK4wB3yhDs,IV,66,106,123,токотоко,RU,IRON,46.3,MvxT79_V7BpuDLdLyiPxArdGhrChwrYaGSBn6sq18hxJF2...,eBU_vMC9DP1oh87heNB4Naukh9jSzmVDrc7gXchWTveUco...,3175,276,1586441782000
1476,-fu37kbTOqrLHwB2SQ5TA6IRt9mn0NiU1V7zNvui5Dccjh4,IV,0,1,20,77ParTiZAN77,RU,IRON,4.8,HGmCt1h_dS6AYsv6e00uzCw6wtj63UYMfopDgJb0KEVQp1...,cohKPHDBIci6o5n_9FzwkJx_ZFYCIHb8aH9pAHiQtePqEF...,4536,63,1586599634000
1477,CaMyu4EK3JiZT_wwRpptWnrG5uWUhqXBdXHZCuTbHvQfviw,IV,22,89,127,СвежаяСопля,RU,IRON,41.2,airn046ozh--CI7XzFc4X23Vdc2LdxAFf8fYcHyOmExgra...,UM8PlbQlCTt_Ru0j810RelDrbZmnGwkaXpmvCdUbKLm_fs...,4248,194,1586461600000
1478,iJDCrbUyKBfk4YJRnzzHu93iICn1ulpmffb4H-5ePWkLuYk,IV,90,6,17,Pablo1991,RU,IRON,26.1,4L8lNMiUmUnrp4DV85hU30fAGqjv_mNJGTggZg9vu868t0...,zNQLek3Mp0FhkcplUCQZRN0i06FFYuObQoRFwwqQMGQU3f...,948,92,1586515840000


In [4]:
df.loc[df['server'].isna()]

,summonerId,tier,leaguePoints,wins,losses,summonerName,server,division,winrate,accountId,puuid,profileIconId,summonerLevel,revisionDate
25,3prE15ujfUSVDnHyPMCg6bcWfQ1Iy5PoP3wpe3HSVzwzDH0,I,470,96,53,tarzaned5,NaN,challengers,64.4,fccggcjWekFzEmqpajw2hPqGWDHBjfZtiHn2uXLManQ1Bp...,gtkdI0NADWYrmSfL4_7t60BjLYy9LGWD9PmvcoNcLD-EiQ...,4066,158,1586472651000
26,NHwqEOuB7n0z2BjlM0ffjLaruKzjcigjw3AgbGzx7YD7t5ur,I,770,87,61,aIan zeng,NaN,challengers,58.8,Y4XbJr-LeL-hwqHMB4OurHH7QE8MQFsbdfdTlC7YJCxnbN...,e8BiU3-7Vlfpe449lvYsdK1dK-yJoiXVEGc7XHZKfNrG9m...,29,58,1586235851000
27,i6ITIi5FJHjyDikheAqcFcTSbgaEA9NLQ_OldL1VuIIiU9AO,I,399,89,56,Levius Cromwell,NaN,challengers,61.4,uGw4eVpiBr3-_eji6H4ajo_GLkJTNeA8W7tFhHS_gukeN3...,m1ICVCqvH6TIpDHQoc053Vp9sVBxyxPa4QnZ0rCmuGnVKD...,4357,155,1586137966000
28,4COMBxqXP3Z9aLd359Jxw9pp1VuypZyguJdihp6MaqvVPhs,I,407,109,90,KatEvolved,NaN,challengers,54.8,9v7WGvMoeIRKm9l2a7ygCw9VgH_1TdTU6P7pkdhXZAdSh8M,54s5K-hnrv9ESmp2jwPJvBGKRv4RFsoQBfZHUighLh1cDP...,3568,329,1586218471000
29,S9beQLLV52wdEHZLfnF7uJin9QDPNVYt-5CF1QW6OK-W6SOY,I,711,103,78,134679316497,NaN,challengers,56.9,rYUcKDH9f5PukR_V_O5PRCp22Z1ejLabc8vJb5eeADgCNp...,C0eh6Pvm8sGe_uCIP_KwHvL6-mf5JuAoHgV29q_SCG1_NA...,4027,62,1586571741000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,w7omwILkhQIuyVD_SBZ04LCFERhOymW-hxK-DT4CCAW21FuM,IV,22,25,48,Lefelwit,NaN,IRON,34.2,lEfJeS7evdkaDLD_abf-ha9rwKzsJrARQeovLgx6tjN0Jz...,lFGIfJoY1qMsP-xwAqDkpu52PF47hieKVxlbWQYNGKULO4...,3879,50,1586545048000
1451,GuOJto15dhZ6v6LfEFlIBGuVraipFJZFwn0C6bU2BMYrflP5,IV,15,41,78,FlsPovyys,NaN,IRON,34.5,hfAsMCaXEgMO8IEwJ6MO2yPJIsCnWGfNLKkGD7ohKdyOn0...,Ne_f7sDHpNmthUzeTjbFX8ckHCCIzvDVDSaUHnAp0TyW4p...,1644,108,1586447283000
1452,I8dtZZydnlztAeFOnm-cpKOMziGa87Dso5NNzHX6L5XbhJk9,IV,61,2,11,Razalith,NaN,IRON,15.4,MsIdd5ADeHfuxrMfFYSw-xYW0SQGUvFghZmcRYHv8Nwa0A...,CGbEYrIO8Bpn9yum4P9f9dgVnG9nIzG-eGJvZ3NXOFwo92...,27,42,1586597351000
1453,nvmkt3-UUBOIKmSuNvG4CFGeToIcbVgzEfH2h326goHlq1Jv,IV,100,8,13,ITripid,NaN,IRON,38.1,-8Qt0OEaNs2JHykPzP_V8r1Y3cA-4G1J_uozNf7-LHODS_...,qG3RM-oXyra4oDicVmL_3LpdrIgwvVdXztKAoe4lbXMxN1...,3800,87,1586110210000


In [7]:
data = [{"key":"NA"}]
pd.DataFrame(data)

,key
0,NA
